In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import time

In [5]:
os.environ['TRANSFORMERS_OFFLINE']="1"

In [6]:
os.environ['TRANSFORMERS_CACHE'] = '/scratch/shareddata/dldata/huggingface-hub-cache/hub'

In [7]:
import torch
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM

## Generate responds

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "gpt2"

In [ ]:
tokenizer = tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name,
                                                         # load_in_8bit=True,
                                                  # torch_dtype=torch.float16,
                                                 device_map="auto")
# model_ref = AutoModelForCausalLMWithValueHead.from_pretrained(model_name,torch_dtype=torch.float16)

In [ ]:
device = model.pretrained_model.device

In [ ]:
# Initialize an empty list to store the sentences
prompts_list = []

# Open the text file
with open('prompts.txt', 'r') as file:
    # Read each line in the file
    for line in file:
        # Add the line (sentence) to the list
        prompts_list.append(line.strip())

# Now 'sentences' contains all the sentences from the file
print( prompts_list[:10])
print(len(prompts_list))

In [ ]:
batch_size = 2
prompt_batches = [prompts_list[i:i + batch_size] for i in range(0, len(prompts_list), batch_size)]
# print(prompt_batches)

In [ ]:
all_responds = []
for batch in prompt_batches:

    # Tokenize the batch
    inputs = tokenizer(batch, padding=True, truncation=True,max_length=30, return_tensors="pt").to(device)

    # Generate outputs
    outputs = model.generate(**inputs)
    
    # Decode the outputs
    responds = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # Collect all generated texts
    all_responds.extend(responds)


In [ ]:
len(all_responds)

In [11]:
from datasets import Dataset

# data_dict = {
#     "prompt": prompts_list,  # List of all prompts
#     "response": all_responds  # Corresponding generated texts
# }

# dataset = Dataset.from_dict(data_dict)
# dataset.save_to_disk("./completion_llama2_7b_chat")

# For native format
loaded_dataset = Dataset.load_from_disk("./completion_llama2_7b_chat")

## Get rewards

In [8]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",                      model="facebook/bart-large-mnli",  batch_size=2
#                       device=0 #use 1 gpu
                     )

In [9]:
classifier.model

BartForSequenceClassification(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): L

In [12]:
sequences_to_classify = loaded_dataset['response']
candidate_labels = ['positive', 'negative']
results = classifier(sequences_to_classify, candidate_labels)


In [13]:
results[:2]

[{'sequence': 'As the sun dipped below the horizon, she whispered Sheila\'s name into the darkness.\n\nThe wind rustled through the trees, carrying the sound of her voice to the ears of the sleeping girl. She stirred, her eyelids flickering open, and she saw her mother standing over her, her face bathed in the soft, golden light of the setting sun.\n\n"Mother?" Sheila said, her voice barely above a whisper.\n\n"Shh, my child," her mother replied, her voice low and soothing. "I\'m here, and I\'ll always be here for you."\n\nSheila smiled, feeling a sense of peace wash over her. She knew that her mother would always be there to protect her, to guide her, and to love her. And as the stars began to twinkle in the sky, she drifted off to sleep, surrounded by the warmth and love of her mother\'s embrace.',
  'labels': ['positive', 'negative'],
  'scores': [0.8609572649002075, 0.1390426903963089]},
 {'sequence': 'In the heart of the bustling city, the old clock tower stood tall and proud, a s

In [14]:
rewards = [result['scores'][0] for result in results]
rewards

[0.8609572649002075,
 0.9228182435035706,
 0.5317773222923279,
 0.8464366793632507,
 0.8121617436408997,
 0.8203584551811218,
 0.930128812789917,
 0.8936605453491211,
 0.6091751456260681,
 0.7915117740631104,
 0.8447045683860779,
 0.5947400331497192,
 0.9344221949577332,
 0.7813522219657898,
 0.6773231625556946,
 0.8200052380561829,
 0.5995565056800842,
 0.7572034001350403,
 0.9054780006408691,
 0.657005250453949,
 0.6353934407234192,
 0.8420088291168213,
 0.9306694269180298,
 0.5814589262008667,
 0.5401087403297424,
 0.8723199367523193,
 0.7645217180252075,
 0.8475704789161682,
 0.952715277671814,
 0.6110256314277649,
 0.5725849866867065,
 0.9297687411308289,
 0.7758700847625732,
 0.8660721182823181,
 0.7207596898078918,
 0.6905559301376343,
 0.7340685725212097,
 0.7655181884765625,
 0.8030501008033752,
 0.8590245246887207,
 0.6460024118423462,
 0.5988187789916992,
 0.9237439632415771,
 0.6969642043113708,
 0.8532393574714661,
 0.9171209335327148,
 0.799035906791687,
 0.77690845727920